In [1]:
import pandas as pd
import numpy as np

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': '1xj78CNKOvswnTZ7_wIfFr-v9Pt3mV5-C'})
downloaded.GetContentFile('con links cargados.csv')

textos=pd.read_csv('con links cargados.csv')

##Analisis exploratorio

In [2]:
textos.groupby("asignaturasDeptos").count().sort_values(by=['asignaturas'], ascending = False)

# df.nunique().to_frame().T

,asignaturas,planificaciones
asignaturasDeptos,,
https://www.fi.uba.ar/institucional/departamentos/electronica/asignaturas,212,212
https://www.fi.uba.ar/institucional/departamentos/computacion/asignaturas,115,115
https://www.fi.uba.ar/institucional/departamentos/gestion/asignaturas,98,98
https://www.fi.uba.ar/institucional/departamentos/energia/asignaturas,94,94
https://www.fi.uba.ar/institucional/departamentos/ingenieria-mecanica/asignaturas,85,85
https://www.fi.uba.ar/institucional/departamentos/ingenieria-quimica/asignaturas,48,46
https://www.fi.uba.ar/institucional/departamentos/agrimensura/asignaturas,47,47
https://www.fi.uba.ar/institucional/departamentos/tecnologia-industrial/asignaturas,45,45
https://www.fi.uba.ar/institucional/departamentos/departamento-de-construcciones-y-estructuras/asignaturas,43,43


In [31]:
textos["planificaciones"]

0      https://cms.fi.uba.ar/uploads/7000_f3e5b5598f.pdf
1      https://cms.fi.uba.ar/uploads/7001_d6c8e9d00f.pdf
2      https://cms.fi.uba.ar/uploads/7002_64d783b823.pdf
3      https://cms.fi.uba.ar/uploads/7003_b7b9d87b24.pdf
4      https://cms.fi.uba.ar/uploads/7004_334c8672cb.pdf
                             ...                        
978    https://cms.fi.uba.ar/uploads/8810_0e4dd94754.pdf
979    https://cms.fi.uba.ar/uploads/8811_80899054e3.pdf
980    https://cms.fi.uba.ar/uploads/8812_a4155b47ac.pdf
981    https://cms.fi.uba.ar/uploads/8813_baf51c5493.pdf
982    https://cms.fi.uba.ar/uploads/8814_213fffa227.pdf
Name: planificaciones, Length: 983, dtype: object

In [ ]:
#asignaturas que no tienen un pdf válido
textos[~textos["planificaciones"].str.contains("cms.fi.uba.ar", na=False)]

In [31]:
textos = textos[textos["planificaciones"].str.contains("cms.fi.uba.ar", na=False)]

##PDF's

###pdfPlumber

In [ ]:
!pip install pdfplumber

In [4]:
import urllib3
import pdfplumber
import io

# https://github.com/jsvine/pdfplumber
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [5]:
#https://stackoverflow.com/questions/62075033/read-pdf-from-url-to-memory-omitting-saving-file-to-local-file

#extrae todo el texto, de un enlace a un pdf (dependiendo del contenido puede tardar entre 0.6 seg a 1 seg)
def extract_pdf_by_url(url):
    http = urllib3.PoolManager()
    temp = io.BytesIO()
    temp.write(http.request("GET", url).data)
    all_text = ''
    with pdfplumber.open(temp) as pdf:
        for pdf_page in pdf.pages:
            single_page_text = pdf_page.extract_text()
            all_text = all_text + '\n' + single_page_text
    return all_text

In [6]:
#dado un pdf filtra la sección desde "objetivos" hasta "bibliografía"

#en caso de querer separarlos en más categorías guardar en sublistas
def extraerInfo(pdfActual):
  titulos = ["OBJETIVOS", "CONTENIDOS MÍNIMOS", "PROGRAMA SINTÉTICO", "PROGRAMA ANALÍTICO", "BIBLIOGRAFÍA"]

  infoGuardada = []
  guardar = False
  for palabra in pdfActual.splitlines():    #https://stackoverflow.com/questions/24237524/how-to-split-a-python-string-on-new-line-characters
    if(palabra == titulos[0]):
      guardar = True
    # else:
    #   print("Error al inicio en:", )
    if(palabra == titulos[-1]):
      guardar = False
    if guardar:
      infoGuardada.append(palabra)

  # print("\n". join(infoGuardada))
  return ("\n". join(infoGuardada))

In [51]:
sample5 = textos#.head(5).copy()
sample5

,asignaturasDeptos,asignaturas,planificaciones,contenido
0,https://www.fi.uba.ar/institucional/departamen...,70.00 Tesis,https://cms.fi.uba.ar/uploads/7000_f3e5b5598f.pdf,NaN
1,https://www.fi.uba.ar/institucional/departamen...,70.01 Geometría Proyectiva,https://cms.fi.uba.ar/uploads/7001_d6c8e9d00f.pdf,NaN
2,https://www.fi.uba.ar/institucional/departamen...,70.02 Geometría Descriptiva,https://cms.fi.uba.ar/uploads/7002_64d783b823.pdf,NaN
3,https://www.fi.uba.ar/institucional/departamen...,70.03 Medios de Representación A,https://cms.fi.uba.ar/uploads/7003_b7b9d87b24.pdf,NaN
4,https://www.fi.uba.ar/institucional/departamen...,70.04 Dibujo Topográfico,https://cms.fi.uba.ar/uploads/7004_334c8672cb.pdf,NaN
...,...,...,...,...
978,https://www.fi.uba.ar/institucional/departamen...,88.10 Caminos,https://cms.fi.uba.ar/uploads/8810_0e4dd94754.pdf,NaN
979,https://www.fi.uba.ar/institucional/departamen...,88.11 Aeropuertos A,https://cms.fi.uba.ar/uploads/8811_80899054e3.pdf,NaN
980,https://www.fi.uba.ar/institucional/departamen...,88.12 Ferrocarriles A,https://cms.fi.uba.ar/uploads/8812_a4155b47ac.pdf,NaN
981,https://www.fi.uba.ar/institucional/departamen...,88.13 Puertos y Vías Navegables A,https://cms.fi.uba.ar/uploads/8813_baf51c5493.pdf,NaN


In [52]:
pd.options.mode.chained_assignment = None
sample5["contenido"] = np.nan

In [53]:
sample5

,asignaturasDeptos,asignaturas,planificaciones,contenido
0,https://www.fi.uba.ar/institucional/departamen...,70.00 Tesis,https://cms.fi.uba.ar/uploads/7000_f3e5b5598f.pdf,NaN
1,https://www.fi.uba.ar/institucional/departamen...,70.01 Geometría Proyectiva,https://cms.fi.uba.ar/uploads/7001_d6c8e9d00f.pdf,NaN
2,https://www.fi.uba.ar/institucional/departamen...,70.02 Geometría Descriptiva,https://cms.fi.uba.ar/uploads/7002_64d783b823.pdf,NaN
3,https://www.fi.uba.ar/institucional/departamen...,70.03 Medios de Representación A,https://cms.fi.uba.ar/uploads/7003_b7b9d87b24.pdf,NaN
4,https://www.fi.uba.ar/institucional/departamen...,70.04 Dibujo Topográfico,https://cms.fi.uba.ar/uploads/7004_334c8672cb.pdf,NaN
...,...,...,...,...
978,https://www.fi.uba.ar/institucional/departamen...,88.10 Caminos,https://cms.fi.uba.ar/uploads/8810_0e4dd94754.pdf,NaN
979,https://www.fi.uba.ar/institucional/departamen...,88.11 Aeropuertos A,https://cms.fi.uba.ar/uploads/8811_80899054e3.pdf,NaN
980,https://www.fi.uba.ar/institucional/departamen...,88.12 Ferrocarriles A,https://cms.fi.uba.ar/uploads/8812_a4155b47ac.pdf,NaN
981,https://www.fi.uba.ar/institucional/departamen...,88.13 Puertos y Vías Navegables A,https://cms.fi.uba.ar/uploads/8813_baf51c5493.pdf,NaN


In [54]:
#añade a cada columna la información extraida anteriormente
total = sample5.shape[0]
for i, link in enumerate(sample5["planificaciones"]):     #enumerate devuelve el indice (un nro.) y lo que se itera
  pdfActual = extract_pdf_by_url(link)
  sample5.iloc[i, 3] = extraerInfo(pdfActual)
  print(i+1,"/",total)

931 / 954
932 / 954
933 / 954
934 / 954
935 / 954
936 / 954
937 / 954
938 / 954
939 / 954
940 / 954
941 / 954
942 / 954
943 / 954
944 / 954
945 / 954
946 / 954
947 / 954
948 / 954
949 / 954
950 / 954
951 / 954
952 / 954
953 / 954
954 / 954


In [56]:
sample5

,asignaturasDeptos,asignaturas,planificaciones,contenido
0,https://www.fi.uba.ar/institucional/departamen...,70.00 Tesis,https://cms.fi.uba.ar/uploads/7000_f3e5b5598f.pdf,OBJETIVOS\nCONTENIDOS MÍNIMOS\n-\nPROGRAMA SIN...
1,https://www.fi.uba.ar/institucional/departamen...,70.01 Geometría Proyectiva,https://cms.fi.uba.ar/uploads/7001_d6c8e9d00f.pdf,OBJETIVOS\nCONTENIDOS MÍNIMOS\n-\nPROGRAMA SIN...
2,https://www.fi.uba.ar/institucional/departamen...,70.02 Geometría Descriptiva,https://cms.fi.uba.ar/uploads/7002_64d783b823.pdf,OBJETIVOS\nRepresentar sobre un plano formas t...
3,https://www.fi.uba.ar/institucional/departamen...,70.03 Medios de Representación A,https://cms.fi.uba.ar/uploads/7003_b7b9d87b24.pdf,OBJETIVOS\n1-GENERALES(cid:13)\n(cid:13)\n1.1-...
4,https://www.fi.uba.ar/institucional/departamen...,70.04 Dibujo Topográfico,https://cms.fi.uba.ar/uploads/7004_334c8672cb.pdf,OBJETIVOS\nIntroducir al alumno en las técnica...
...,...,...,...,...
978,https://www.fi.uba.ar/institucional/departamen...,88.10 Caminos,https://cms.fi.uba.ar/uploads/8810_0e4dd94754.pdf,OBJETIVOS\nLa asignatura se centra en el trans...
979,https://www.fi.uba.ar/institucional/departamen...,88.11 Aeropuertos A,https://cms.fi.uba.ar/uploads/8811_80899054e3.pdf,OBJETIVOS\nIntroducir al estudiante de grado e...
980,https://www.fi.uba.ar/institucional/departamen...,88.12 Ferrocarriles A,https://cms.fi.uba.ar/uploads/8812_a4155b47ac.pdf,OBJETIVOS\nEl objetivo primordial es el de pro...
981,https://www.fi.uba.ar/institucional/departamen...,88.13 Puertos y Vías Navegables A,https://cms.fi.uba.ar/uploads/8813_baf51c5493.pdf,OBJETIVOS\nProporcionar a los alumnos un conoc...


In [57]:
markdownFile = []

for i in range(sample5.shape[0]):
  markdownFile.append("### "+sample5.iloc[i, 1])  #nombre materia
  markdownFile.append("\n\n[link al pdf]("+sample5.iloc[i, 2]+")\n\n\n")      #link al pdf
  markdownFile.append(sample5.iloc[i, 3])   #planificacion materia
  markdownFile.append("\n\n")

In [58]:
plainMarkdown = "".join(markdownFile)
plainMarkdown = plainMarkdown.replace("(cid:13", "")
plainMarkdown = plainMarkdown.replace("(cid:9", "")
#expresion regular     \(cid:[0-9]{1,}\)

In [59]:
print(plainMarkdown)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [60]:
with open("contenidos.md", "w") as text_file:
    text_file.write(plainMarkdown)

#luego de escribirlo buscar   \n-\n   y reemplazar por \n    en notepad++   para que el formato markdown no flashee
#reemplazar \n por \n\n
#reemplazar "[0-9] de [0-9]" por nada (con expresiones regulares)

el formato no es modificable (solo desde notapad++, cambiarlo a html no hace mucha diferencia) sin embargo **es un proyecto sencillo**

A hacer
* terminar de parsear todos los pdfs, quitar aquellos que no cumplen con el formato, en todo caso dejarlos en un df aparte e ir abriendolos para ver en que se diferencian
* https://dillinger.io/ para testear markdown de github


#####tests

In [16]:
pdfs = ["https://cms.fi.uba.ar/uploads/7000_f3e5b5598f.pdf","https://cms.fi.uba.ar/uploads/7001_d6c8e9d00f.pdf","https://cms.fi.uba.ar/uploads/7002_64d783b823.pdf","https://cms.fi.uba.ar/uploads/7003_b7b9d87b24.pdf","https://cms.fi.uba.ar/uploads/7004_334c8672cb.pdf","https://cms.fi.uba.ar/uploads/8810_0e4dd94754.pdf","https://cms.fi.uba.ar/uploads/8811_80899054e3.pdf","https://cms.fi.uba.ar/uploads/8812_a4155b47ac.pdf","https://cms.fi.uba.ar/uploads/8813_baf51c5493.pdf","https://cms.fi.uba.ar/uploads/8814_213fffa227.pdf"]

In [17]:
sample5.iloc[0, 1]

'70.00 Tesis'

###Tika


In [ ]:
!pip install tika

In [ ]:
from tika import parser

http = urllib3.PoolManager()
temp = io.BytesIO()
temp.write(http.request("GET", url).data)

rawText = parser.from_file(temp)

rawList = rawText['content'].splitlines()


###PyPDF2


In [ ]:
!pip install PyPDF2

In [22]:
import requests, PyPDF2
from io import BytesIO

url = 'https://cms.fi.uba.ar/uploads/7008_004dc77220.pdf'
response = requests.get(url)
my_raw_data = response.content

with BytesIO(my_raw_data) as data:
    read_pdf = PyPDF2.PdfFileReader(data)

    for page in range(read_pdf.getNumPages()):
        print(read_pdf.getPage(page).extractText())


Planificaciones7008 - Topografía IDocente responsable: PRADELLI ALBERTO1 de 57008 - Topografía IPLANIFICACIONESActualización: 1ºC/2016
OBJETIVOS1- Integrar conocimientos básicos de matemática, física, geometría y dibujo para su aplicación a un findeterminado. 2 - Introducir al alumno en el uso del instrumental y el conocimiento de las técnicas de medición ycálculo necesarias para realizar específicas de relevamiento planialtimétrico de hechos existentes, confecciónde planos topográficos y replanteo de obras.CONTENIDOS MÍNIMOS-PROGRAMA SINTÉTICO-Introducción. -Plano topográfico. Escalas. -Sistemas de Medición Regular. - Teoría de los errores. -Día nolaborable. -Teoría de los errores. -Errores, criterios de Gauss. -Errores, criterios de Chauvenet. -Video,Proyección. -Medición de ángulos con Teodolito. -Teodolito, El Anteojo, Sistemas de Graduación. -Medición deángulos verticales. -Medición directa de distancias. -Medición estadística de distancias. -Medición parláctica.Medición electróni